In [14]:
# Import necessary libraries
import os
import sys

# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))
import pandas as pd
from load_data import fetch_data
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans



In [5]:

# SQL query to fetch data
query = "SELECT * FROM xdr_data ;"  

# Load the data
df = fetch_data(query)

d:\Kifya_training\Week 2\Telecom-User-Analysis-and-Dashboard-Development\scripts\load_data.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [ ]:

# Aggregate the metrics
aggregated_metrics = df.groupby('MSISDN/Number').agg({
    'Dur. (ms)': 'sum',  # Total session duration per customer
    'Total DL (Bytes)': 'sum',  # Total download traffic per customer
    'Total UL (Bytes)': 'sum',  # Total upload traffic per customer
}).reset_index()



In [7]:
# Calculate session frequency
session_frequency = df['MSISDN/Number'].value_counts().reset_index()
session_frequency.columns = ['MSISDN/Number', 'Session Frequency']




In [8]:
# Merge session frequency with other metrics
aggregated_metrics = pd.merge(aggregated_metrics, session_frequency, on='MSISDN/Number')

# Calculate total traffic (Download + Upload)
aggregated_metrics['Total Traffic (Bytes)'] = aggregated_metrics['Total DL (Bytes)'] + aggregated_metrics['Total UL (Bytes)']



In [9]:
# Sort the aggregated data by each metric for ranking
top_10_duration = aggregated_metrics.sort_values(by='Dur. (ms)', ascending=False).head(10)
top_10_traffic = aggregated_metrics.sort_values(by='Total Traffic (Bytes)', ascending=False).head(10)
top_10_frequency = aggregated_metrics.sort_values(by='Session Frequency', ascending=False).head(10)



In [10]:
# Display the top 10 customers for each metric
print("Top 10 Customers by Session Duration:")
print(top_10_duration)

print("\nTop 10 Customers by Total Traffic:")
print(top_10_traffic)

print("\nTop 10 Customers by Session Frequency:")
print(top_10_frequency)

Top 10 Customers by Session Duration:
       MSISDN/Number   Dur. (ms)  Total DL (Bytes)  Total UL (Bytes)  \
13180   3.362578e+10  18553754.0      7.770043e+09       729577380.0   
6437    3.361489e+10   9966898.0      8.156743e+09       689483001.0   
92923   3.376054e+10   9279434.0      7.811295e+09       703478581.0   
13526   3.362632e+10   8791927.0      7.301517e+09       669650721.0   
65118   3.366716e+10   8744914.0      5.052068e+09       566326364.0   
50281   3.366284e+10   6614270.0      3.593674e+09       406424372.0   
57160   3.366469e+10   6288730.0      5.279827e+09       402180804.0   
666     3.360313e+10   6287761.0      4.540607e+09       435587631.0   
66029   3.366746e+10   5649882.0      2.334299e+09       266222647.0   
92577   3.376041e+10   5321667.0      6.610852e+09       521518890.0   

       Session Frequency  Total Traffic (Bytes)  
13180                 17           8.499621e+09  
6437                  17           8.846226e+09  
92923              

In [12]:


# Select the metrics for normalization
metrics_to_normalize = ['Session Frequency', 'Dur. (ms)', 'Total Traffic (Bytes)']

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the metrics
normalized_data = scaler.fit_transform(aggregated_metrics[metrics_to_normalize])

# Convert normalized data back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=metrics_to_normalize)

# Add the MSISDN/Number column to the normalized DataFrame for reference
normalized_df['MSISDN/Number'] = aggregated_metrics['MSISDN/Number']




In [13]:
# Display the first few rows of the normalized data
print("Normalized Data:")
print(normalized_df.head())

Normalized Data:
   Session Frequency  Dur. (ms)  Total Traffic (Bytes)  MSISDN/Number
0           0.000000   0.005908               0.095931   3.360100e+10
1           0.000000   0.009387               0.014026   3.360100e+10
2           0.000000   0.006892               0.063851   3.360100e+10
3           0.000000   0.002304               0.044148   3.360101e+10
4           0.058824   0.001615               0.161598   3.360101e+10


In [15]:


# Initialize the K-Means model with k=3
kmeans = KMeans(n_clusters=3, random_state=42)

# Fit the model on the normalized metrics
kmeans.fit(normalized_data)

# Add the cluster labels to the original data
aggregated_metrics['Cluster'] = kmeans.labels_

# Display the first few rows with cluster assignments
print("Aggregated Metrics with Clusters:")
print(aggregated_metrics.head())

# Display the number of customers in each cluster
print("\nNumber of Customers in Each Cluster:")
print(aggregated_metrics['Cluster'].value_counts())


d:\Kifya_training\Week 2\Telecom-User-Analysis-and-Dashboard-Development\venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\Kifya_training\Week 2\Telecom-User-Analysis-and-Dashboard-Development\venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Yoni\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yoni\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child

Aggregated Metrics with Clusters:
   MSISDN/Number  Dur. (ms)  Total DL (Bytes)  Total UL (Bytes)  \
0   3.360100e+10   116720.0      8.426375e+08        36053108.0   
1   3.360100e+10   181230.0      1.207552e+08        36104459.0   
2   3.360100e+10   134969.0      5.566597e+08        39306820.0   
3   3.360101e+10    49878.0      4.019932e+08        20327526.0   
4   3.360101e+10    37104.0      1.363130e+09        94280527.0   

   Session Frequency  Total Traffic (Bytes)  Cluster  
0                  1           8.786906e+08        2  
1                  1           1.568596e+08        2  
2                  1           5.959665e+08        2  
3                  1           4.223207e+08        2  
4                  2           1.457411e+09        0  

Number of Customers in Each Cluster:
Cluster
2    81158
0    21630
1     4068
Name: count, dtype: int64


In [16]:
# Define the metrics for analysis
metrics = ['Dur. (ms)', 'Total Traffic (Bytes)', 'Session Frequency']

# Compute summary statistics for each cluster
cluster_summary = aggregated_metrics.groupby('Cluster')[metrics].agg(['min', 'max', 'mean', 'sum'])

# Display the summary statistics
print("Cluster Summary Statistics:")
print(cluster_summary)


Cluster Summary Statistics:
        Dur. (ms)                                           \
              min         max           mean           sum   
Cluster                                                      
0         18235.0   3174824.0  213600.528202  4.620179e+09   
1         46896.0  18553754.0  569329.970747  2.316034e+09   
2          7142.0   1573420.0  106984.202543  8.682624e+09   

        Total Traffic (Bytes)                                            \
                          min           max          mean           sum   
Cluster                                                                   
0                2.842182e+08  1.873330e+09  1.125421e+09  2.434285e+13   
1                1.085994e+09  8.846226e+09  2.280321e+09  9.276344e+12   
2                3.324901e+07  9.507607e+08  4.955057e+08  4.021425e+13   

        Session Frequency                       
                      min max      mean    sum  
Cluster                                         
0